## Escalamiento de los datos

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline

from IPython.display import display


<div class="alert alert-success">
  Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.
</div>

Existen distintas estrategias de escalamiento de tus features, pero **la más común es la estandarización** donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [2]:
X = pd.read_csv('./data/transformed/peliculas_opening.csv')
y = X['worldwide_gross']
X.drop(columns=['worldwide_gross', 'movie_title'],inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [3]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [4]:
display(scaler.mean_)
display(scaler.scale_)

array([5.15326093e+07, 5.53647347e+07, 6.33578635e+00, 1.74889508e+07,
       2.22707257e+03, 1.10936499e+02, 2.00601662e+03, 1.36061537e+04,
       2.16150354e+00])

array([4.61777048e+07, 9.24999653e+07, 1.03981055e+00, 2.25023553e+07,
       1.19000043e+03, 2.01235679e+01, 5.47340669e+00, 2.34057269e+04,
       2.50195735e-01])

In [5]:
display(X.values)
display(scaler.transform(X_train))

array([[4.25000000e+08, 2.37000000e+08, 7.90000000e+00, ...,
        2.00900000e+03, 4.83400000e+03, 1.78000000e+00],
       [3.00000000e+08, 3.00000000e+08, 7.10000000e+00, ...,
        2.00700000e+03, 4.83500000e+04, 2.35000000e+00],
       [3.00000000e+08, 2.45000000e+08, 6.80000000e+00, ...,
        2.01500000e+03, 1.17000000e+04, 2.35000000e+00],
       ...,
       [1.05000000e+07, 1.00000000e+07, 6.50000000e+00, ...,
        2.00700000e+03, 2.03120000e+04, 2.35000000e+00],
       [1.01000000e+07, 3.71233913e+07, 5.30000000e+00, ...,
        2.01000000e+03, 1.12100000e+03, 2.12668290e+00],
       [1.01000000e+07, 1.10000000e+07, 5.80000000e+00, ...,
        2.00700000e+03, 1.52600000e+03, 1.85000000e+00]])

array([[-0.72616449, -0.40394323,  1.21581152, ...,  0.72777029,
        -0.06930585, -1.24503937],
       [ 2.45718992,  1.18524656,  2.17752517, ...,  1.45857657,
         0.76399449,  0.75339598],
       [-0.6395426 , -0.36069997, -0.61144441, ..., -1.09924541,
        -0.48663961, -1.24503937],
       ...,
       [-0.31471052, -0.18772693, -0.22675895, ...,  1.82397972,
        -0.37273586,  0.75339598],
       [ 1.11455065,  0.51497603,  2.08135381, ..., -1.09924541,
        -0.30271026,  0.75339598],
       [-0.27139957, -0.17691612,  0.7349547 , ...,  0.54506872,
        -0.55704973,  0.75339598]])

In [6]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [7]:
model = Lasso()
model_scaled = Lasso()

model.fit(X_train,y_train)
model_scaled.fit(X_train_scaled,y_train)

print('{:.2%}'.format(model.score(X_test,y_test)))
print('{:.2%}'.format(model_scaled.score(X_test_scaled,y_test)))

79.01%
79.01%


<div class="alert alert-success">
  Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí. 
</div>

## Simplificar las transformaciones con pipelines

<div class="alert alert-info">
  Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar <code> sklearn.pipeline.make_pipeline </code> que permite encadenar transformaciones a tus modelos.
</div>

In [8]:
model_scaled = make_pipeline(StandardScaler(),
                             Lasso() )
model_scaled.fit(X_train, y_train)
'{:.2%}'.format(model_scaled.score(X_test, y_test))

'79.01%'

## Crear nuevas features de forma automática

In [9]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [10]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
A_transformed = transformer.fit_transform(A)

display(A.shape, A)
display(transformer.get_feature_names())
display(A_transformed.shape, A_transformed)

(3, 2)

array([[0, 1],
       [2, 3],
       [4, 5]])

['1', 'x0', 'x1', 'x0^2', 'x0 x1', 'x1^2']

(3, 6)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

``PolynomialFeatures`` transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [11]:
X.shape, transformer.fit_transform(X).shape

((2247, 9), (2247, 55))

In [12]:
model_poly = make_pipeline( PolynomialFeatures(2),
                            Lasso())
model_poly.fit(X_train, y_train)
'{:.2%}'.format(model_poly.score(X_test, y_test))

'80.19%'

## Crear features categóricas

<div class="alert alert-success">
  En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.
</div>

Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se. 

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

_**Encoding one-hot**_

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [13]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [14]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido ``pd.get_dummies``. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [15]:
movies_obj = pd.read_csv('./data/transformed/peliculas_obj.csv')
X = pd.read_csv('./data/transformed/peliculas_opening.csv')

movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      17
language            39
country             55
genres             809
actor_1_name      1702
director_name     1966
actor_2_name      2506
actor_3_name      2915
plot_keywords     3990
movie_title       4061
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

## Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [16]:
categoricals = movies_obj[['movie_title', 'actor_1_name', 'director_name']]
display(categoricals.shape) 
display(categoricals.head(2))

(4061, 3)

,movie_title,actor_1_name,director_name
0,Avatar,CCH Pounder,James Cameron
1,Star Wars: Episode VII - The Force Awakens ...,Doug Walker,Doug Walker


In [17]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247 entries, 0 to 2246
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   movie_title                2247 non-null   object 
 1   production_budget          2247 non-null   int64  
 2   budget                     2247 non-null   float64
 3   imdb_score                 2247 non-null   float64
 4   opening_gross              2247 non-null   float64
 5   screens                    2247 non-null   float64
 6   duration                   2247 non-null   float64
 7   title_year                 2247 non-null   float64
 8   cast_total_facebook_likes  2247 non-null   int64  
 9   aspect_ratio               2247 non-null   float64
 10  worldwide_gross            2247 non-null   int64  
dtypes: float64(7), int64(3), object(1)
memory usage: 193.2+ KB


In [18]:
bin_encoder = pd.merge(X, categoricals)

bin_encoder.drop(columns=['movie_title', 'worldwide_gross'], inplace=True)
X.drop(columns=['movie_title', 'worldwide_gross'], inplace=True)

display(bin_encoder.shape)
bin_encoder.head(2)

(2247, 11)

,production_budget,budget,imdb_score,opening_gross,screens,duration,title_year,cast_total_facebook_likes,aspect_ratio,actor_1_name,director_name
0,425000000,237000000.0,7.9,77025481.0,3452.0,178.0,2009.0,4834,1.78,CCH Pounder,James Cameron
1,300000000,300000000.0,7.1,139802190.0,4362.0,169.0,2007.0,48350,2.35,Johnny Depp,Gore Verbinski


In [19]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name', 'director_name'])
encoder

BinaryEncoder(cols=['actor_1_name', 'director_name'], drop_invariant=False,
              handle_missing='value', handle_unknown='value', mapping=None,
              return_df=True, verbose=0)

In [20]:
bin_encoder = encoder.fit_transform(bin_encoder)
bin_encoder.shape

(2247, 32)

In [21]:
Xb_train, Xb_test, y_train, y_test = train_test_split(bin_encoder, y, random_state=1)
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

model_binenc = Lasso()
model = Lasso()

model_binenc.fit(Xb_train, y_train)
model.fit(X_train, y_train)

print('{:.2%}'.format(model_binenc.score(Xb_test, y_test)))
print('{:.2%}'.format(model.score(X_test, y_test)))

78.77%
79.01%


Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.

## Conocimiento experto

<div class="alert alert-success">
  Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**. <br>
  Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).** <br>
  Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.
</div>

## Más datos de calidad

Nada le gana conseguir más datos que sean encodeables en features de calidad.

_**Píramide de Maslow del Machine Learning**_

<img src="./img/maslow.png">

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.

Puedes mejorar considerablemente nuestra predicción?